# 🏰 Gatekeeper CTF Challenge

## Scenario

A small village uses an **AI-powered Gatekeeper** to decide who is allowed to enter.  
The Gatekeeper is a machine learning model. It was trained on records of villagers and learned to tell apart regular citizens from thieves based on their behavior.

Every person in the dataset has four behavioral features:

| Feature | Description |
|---|---|
| `Night_Activity` | How active the person is during the night |
| `Trust_Index` | How much the community trusts this person |
| `Contribution` | How much this person contributes positively to the village |
| `Conflict_Score` | How many conflicts this person has been involved in |

Each record also has a **Label** that tells us who the person really is:

- **Label = 0 → Citizen** (a normal, innocent villager)
- **Label = 1 → Thief** (a known criminal)

The model reads the four features and predicts whether someone is a **Citizen (0)** or a **Thief (1)**.

---

## 🎯 Your Mission

One person in the dataset — **ID = 10** — is a known thief (Label = 1).  
The Gatekeeper correctly identifies them as a thief right now.

Your job is to **tamper with the training data** so that after the model retrains, it predicts ID = 10 as a **Citizen (0)** instead of a Thief.

Think of it as poisoning the data the model learns from, so it makes the wrong decision.

**Rules:**
- You may tamper with the data in **one of two ways**:
  - Use the **interactive table** provided in this notebook to edit values and save
  - Open `gatekeeper_dataset.csv` directly, make your changes, and save it as `tampered_data.csv`
- **Writing code to manipulate the data is not allowed**
- The model retrains on whatever is in `tampered_data.csv` at submission time
- A successful attack = Thief (ID 10) predicted as **Citizen (0)**
- 📤 **Submit only the `tampered_data.csv` file** — do NOT submit the notebook

---

## Pipeline Overview

```
Raw Data → Load CSV → Train Model → Predict Label for ID 10
                 ↑

          [You tamper here]```

## Step 1: Install Dependencies

Install all required Python packages for this notebook:
- `ipywidgets` — interactive UI controls (sliders, buttons)
- `ipydatagrid` — editable spreadsheet-style data table
- `scikit-learn` — machine learning (Logistic Regression, StandardScaler)
- `pandas` — data loading and manipulation

> Run this cell first before any other cell.

In [2]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


## Step 2: Load the Gatekeeper Dataset

This cell loads the dataset from `gatekeeper_dataset.csv` into memory.

The dataset is a simple table where each row represents one person. Here is what it contains:

- **ID** — a unique number to identify each person
- **Night_Activity, Trust_Index, Contribution, Conflict_Score** — the four behavioral features the model uses to make its decision
- **Label** — the true identity of the person:
  - `0` = Citizen (innocent villager)
  - `1` = Thief (known criminal)

The model is trained on all the rows in this table. It looks at the four features of each person and learns the pattern that separates citizens from thieves. Once trained, it uses that pattern to predict the label for ID = 10.

> 🔒 **Do not modify the code cell below.** It loads the dataset and must run exactly as written.

In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Load dataset
df = pd.read_csv("gatekeeper_dataset.csv")

## Step 3: Edit the Dataset — Your Attack Surface

You have two options to tamper with the data:

**Option A — Use the interactive table below:**
1. Click any cell in the table and type a new value
2. Modify feature values to influence how the model classifies each record
3. When done, click **Save as CSV** — this saves your tampered version to `tampered_data.csv`

**Option B — Edit the file directly:**
1. Open `gatekeeper_dataset.csv` in any spreadsheet app or text editor
2. Make your changes
3. Save the file as `tampered_data.csv` in this folder

> ⚠️ **Writing code to manipulate the data is not allowed.** Only the two methods above are permitted.

**Buttons:**
| Button | Action |
|---|---|

| **Reset to Original** | Restores the table to the unmodified dataset || **Save as CSV** | Saves the current (possibly tampered) table to a CSV file |

## 📤 Submission

Once you have tampered with the dataset and are confident the Gatekeeper will misclassify the thief:

1. Make sure your tampered data is saved as `tampered_data.csv` (via the table or by editing the file directly)
2. Run the cell below to submit it to the evaluation server and get your result instantly

> ⚠️ Do NOT submit the notebook. Only the CSV file will be evaluated by the automated judge.

> 🔒 **Do not modify the code cell below.** It sets up the interactive table and the Save/Reset buttons — run it as-is.

In [4]:
import ipywidgets as widgets
from ipydatagrid import DataGrid
from IPython.display import display, clear_output, HTML

# Working copy of the dataset
edit_df = df.copy()

# --- Editable DataGrid ---
grid = DataGrid(
    edit_df,
    editable=True,
    layout={"height": "300px"},
    column_widths={
        "ID": 50,
        "Night_Activity": 120,
        "Trust_Index": 110,
        "Contribution": 110,
        "Conflict_Score": 120,
        "Label": 70,
    }
)

# --- Output area ---
output = widgets.Output()

# --- Buttons ---
reset_btn = widgets.Button(
    description="Reset to Original",
    button_style="warning",
    icon="refresh",
    layout=widgets.Layout(width="180px", margin="10px 4px")
)

save_btn = widgets.Button(
    description="Save as CSV",
    button_style="success",
    icon="download",
    layout=widgets.Layout(width="160px", margin="10px 4px")
)

save_filename = widgets.Text(
    value="tampered_data.csv",
    description="Filename:",
    style={"description_width": "70px"},
    layout=widgets.Layout(width="280px", margin="10px 4px")
)

# --- Button callbacks ---
def on_reset(b):
    with output:
        clear_output()
        display(HTML("<p style='color:gray;font-family:Arial;'>Table reset to original data.</p>"))
    grid.data = df.copy()

def on_save(b):
    with output:
        clear_output()
        fname = save_filename.value.strip() or "tampered_data.csv"
        if not fname.endswith(".csv"):
            fname += ".csv"
        grid.data.to_csv(fname, index=False)
        display(HTML(f"<p style='color:green;font-family:Arial;font-weight:bold;'>✅ Saved to <code>{fname}</code></p>"))

reset_btn.on_click(on_reset)
save_btn.on_click(on_save)

# --- Layout ---
display(widgets.HTML("<h3 style='font-family:Arial'>✏️ Editable Gatekeeper Dataset — click any cell to edit</h3>"))
display(grid)
display(widgets.HBox([reset_btn, save_btn, save_filename]))
display(output)

HTML(value="<h3 style='font-family:Arial'>✏️ Editable Gatekeeper Dataset — click any cell to edit</h3>")

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, column_widths={'ID': 50, 'Night_Acti…

Output()

## 📤 Submit Your Attack

Once you have made your changes and saved the file, run the cell below.

It will send your tampered dataset to the Gatekeeper evaluation server. The server will retrain the model on your data and test whether the thief gets past the gate.

You will get an instant response telling you whether your attack worked, along with how confident the model was in its decision.

> 🔒 **Do not modify the code cell below.** It handles the submission to the evaluation server — run it as-is.

In [5]:
import requests
import os

# ── Configuration ─────────────────────────────────────────────────────────────
API_URL  = "http://localhost:8000/challenge-1"
CSV_FILE = "tampered_data.csv"

# ── Submit ────────────────────────────────────────────────────────────────────
if not os.path.exists(CSV_FILE):
    print(f"❌ '{CSV_FILE}' not found. Save your tampered data first using the table above.")
else:
    with open(CSV_FILE, "rb") as f:
        response = requests.post(API_URL, files={"file": (CSV_FILE, f, "text/csv")})

    result = response.json()

    print("=" * 50)
    if result.get("result") == "success":
        print("✅  CHALLENGE SOLVED!")
        print(f"🏴  Flag : {result['flag']}")
    else:
        print("❌  Not quite right. Keep trying!")

    print(f"\n💬  {result.get('message', '')}")
    print(f"\n📊  Model Confidence:")
    print(f"    Citizen : {result.get('citizen_confidence', 'N/A')}")
    print(f"    Thief   : {result.get('thief_confidence',   'N/A')}")
    print("=" * 50)

✅  CHALLENGE SOLVED!
🏴  Flag : CTF{p01s0n_th3_w3ll_g4t3_f4lls}

💬  The Gatekeeper has been fooled! The thief is classified as a Citizen.

📊  Model Confidence:
    Citizen : 0.8947
    Thief   : 0.1053
